In [15]:
# import libraries
import numpy as np
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score, confusion_matrix
from sklearn.svm import SVC
from sklearn import preprocessing
import pickle
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss

## Loading dataset, for our purposes, simple digit dataset with 10 classes

In [7]:
data = load_digits()
X = data.data
y = data.target
print(X.shape)
print(y.shape)

(1797, 64)
(1797,)


## Initial testing of SVM algorithm on data

In [17]:
# Implementing 10-fold cross validation to test model
cv = KFold(n_splits=10, shuffle=False, random_state=234)

# lists to store model metrics for analysis
store_train_acc = []
store_acc = []

store_precs = []
store_recs = []
store_f1s = []
store_hloss = []

# Implementing 10-fold here
for train_index, test_index in cv.split(X,y):
    # split data to 9 folds training and 1 fold testing
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # implementing preprocessing scheme on only training data
    # fitting testing data to preprocessing method
    normalizer = preprocessing.StandardScaler().fit(X_train)
    X_train = normalizer.transform(X_train)
    X_test = normalizer.transform(X_test)
    
    # SVM classifier algorithm selected
    clf = SVC()
    clf.fit(X_train, y_train)
    
    # compute different metrics, print for each fold, store for analysis
    y_pred_train = clf.predict(X_train)
    y_pred = clf.predict(X_test)
    
    train_acc = accuracy_score(y_train, y_pred_train)
    acc = accuracy_score(y_test, y_pred)
    print('Training accuracy is: {}, testing accuracy is: {}'.format(train_acc,acc))
    
    precs = precision_score(y_test, y_pred, average='macro')
    recs = recall_score(y_test, y_pred, average='macro')
    f1s = f1_score(y_test, y_pred, average='macro')
    hloss = hamming_loss(y_test, y_pred)
    print('Precision score: {}'.format(precs))
    print('Recall score: {}'.format(recs))
    print('F1 score: {}'.format(f1s))
    print('Hamming loss: {}'.format(hloss))
    
    store_train_acc.append(train_acc)
    store_acc.append(acc)
    
    store_precs.append(precs)
    store_recs.append(recs)
    store_f1s.append(f1s)
    store_hloss.append(hloss)

# Compute and print average and standard deviation of metrics
print('Average training accuracy: {}, standard deviation: {}'.format(np.mean(np.array(store_train_acc)),np.std(np.array(store_train_acc))))
print('Average testing accuracy: {}, standard deviation: {}'.format(np.mean(np.array(store_acc)),np.std(np.array(store_acc))))
print('Average precision score: {}, standard deviation: {}'.format(np.mean(np.array(store_precs)),np.std(np.array(store_precs))))
print('Average recall score: {}, standard deviation: {}'.format(np.mean(np.array(store_recs)),np.std(np.array(store_recs))))
print('Average f1 score: {}, standard deviation: {}'.format(np.mean(np.array(store_f1s)),np.std(np.array(store_f1s))))
print('Average hamming loss score: {}, standard deviation: {}'.format(np.mean(np.array(store_hloss)),np.std(np.array(store_hloss))))


Training accuracy is: 0.9981447124304267, testing accuracy is: 0.9444444444444444
Precision score: 0.952705314009662
Recall score: 0.9414181286549708
F1 score: 0.9410935310742756
Hamming loss: 0.05555555555555555
Training accuracy is: 0.9962894248608535, testing accuracy is: 0.9944444444444445
Precision score: 0.9944444444444445
Recall score: 0.99375
F1 score: 0.9939170506912444
Hamming loss: 0.005555555555555556
Training accuracy is: 0.9969078540507111, testing accuracy is: 0.9777777777777777
Precision score: 0.9813636363636362
Recall score: 0.9777433780529755
F1 score: 0.9786715328178743
Hamming loss: 0.022222222222222223
Training accuracy is: 0.9962894248608535, testing accuracy is: 0.9444444444444444
Precision score: 0.9448099415204678
Recall score: 0.9427760577915377
F1 score: 0.9426126355538121
Hamming loss: 0.05555555555555555
Training accuracy is: 0.9962894248608535, testing accuracy is: 0.9777777777777777
Precision score: 0.9802130325814536
Recall score: 0.9780701754385965
F1 

## Hyperparameter tuning 

In [18]:
# if results are not enough, or we just want a more robust model,
# perform hyperparameter tuning. I will use exhaustive gridsearch

# Implementing 10-fold cross validation to test model
cv = KFold(n_splits=10, shuffle=False, random_state=234)
inner_cv = KFold(n_splits=10, shuffle=False, random_state=234)

# lists to store model metrics for analysis
store_train_acc = []
store_acc = []

store_precs = []
store_recs = []
store_f1s = []
store_hloss = []

# Implementing 10-fold here
for train_index, test_index in cv.split(X,y):
    # split data to 9 folds training and 1 fold testing
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # implementing preprocessing scheme on only training data
    # fitting testing data to preprocessing method
    normalizer = preprocessing.StandardScaler().fit(X_train)
    X_train = normalizer.transform(X_train)
    X_test = normalizer.transform(X_test)
    
    params = {'C': np.linspace(1,1000,10,dtype=int), 'kernel':['rbf','linear','sigmoid'],
             'gamma': np.linspace(0.001,0.1,10)}
    
    # SVM classifier algorithm selected
    clf = GridSearchCV(estimator=SVC(), param_grid=params, cv=inner_cv, scoring='accuracy', n_jobs=4)
    clf.fit(X_train, y_train)
    
    # compute different metrics, print for each fold, store for analysis
    y_pred_train = clf.predict(X_train)
    y_pred = clf.predict(X_test)
    
    train_acc = accuracy_score(y_train, y_pred_train)
    acc = accuracy_score(y_test, y_pred)
    print('Training accuracy is: {}, testing accuracy is: {}'.format(train_acc,acc))
    
    precs = precision_score(y_test, y_pred, average='macro')
    recs = recall_score(y_test, y_pred, average='macro')
    f1s = f1_score(y_test, y_pred, average='macro')
    hloss = hamming_loss(y_test, y_pred)
    print('Precision score: {}'.format(precs))
    print('Recall score: {}'.format(recs))
    print('F1 score: {}'.format(f1s))
    print('Hamming loss: {}'.format(hloss))
    
    store_train_acc.append(train_acc)
    store_acc.append(acc)
    
    store_precs.append(precs)
    store_recs.append(recs)
    store_f1s.append(f1s)
    store_hloss.append(hloss)

# Compute and print average and standard deviation of metrics
print('Average training accuracy: {}, standard deviation: {}'.format(np.mean(np.array(store_train_acc)),np.std(np.array(store_train_acc))))
print('Average testing accuracy: {}, standard deviation: {}'.format(np.mean(np.array(store_acc)),np.std(np.array(store_acc))))
print('Average precision score: {}, standard deviation: {}'.format(np.mean(np.array(store_precs)),np.std(np.array(store_precs))))
print('Average recall score: {}, standard deviation: {}'.format(np.mean(np.array(store_recs)),np.std(np.array(store_recs))))
print('Average f1 score: {}, standard deviation: {}'.format(np.mean(np.array(store_f1s)),np.std(np.array(store_f1s))))
print('Average hamming loss score: {}, standard deviation: {}'.format(np.mean(np.array(store_hloss)),np.std(np.array(store_hloss))))


Training accuracy is: 1.0, testing accuracy is: 0.9388888888888889
Precision score: 0.948391812865497
Recall score: 0.9364181286549709
F1 score: 0.9361604808973232
Hamming loss: 0.06111111111111111
Training accuracy is: 1.0, testing accuracy is: 0.9944444444444445
Precision score: 0.9941176470588236
Recall score: 0.99375
F1 score: 0.9937438905180841
Hamming loss: 0.005555555555555556
Training accuracy is: 1.0, testing accuracy is: 0.9833333333333333
Precision score: 0.9854761904761904
Recall score: 0.983625730994152
F1 score: 0.9840189090189089
Hamming loss: 0.016666666666666666
Training accuracy is: 1.0, testing accuracy is: 0.9833333333333333
Precision score: 0.9830065359477125
Recall score: 0.9832989336085312
F1 score: 0.9828435157846924
Hamming loss: 0.016666666666666666
Training accuracy is: 1.0, testing accuracy is: 0.9888888888888889
Precision score: 0.9897368421052631
Recall score: 0.9888888888888889
F1 score: 0.989018909018909
Hamming loss: 0.011111111111111112
Training accura

In [19]:
clf.best_params_

{'C': 112, 'gamma': 0.012, 'kernel': 'rbf'}

## final training and saving model

In [20]:
# we completed the hyperparameter tuning and found the
# best parameters above
# There was only a slight improvement in accuracy but 
# these efforts are necessary. Caveat: the dataset was very
# simple to train so only modest improvements were seen. If
# computational resources allow, I would run gridsearch through
# an even wider set of parameters (all on local now)

normalizer = preprocessing.StandardScaler().fit(X)
X_train = normalizer.transform(X)
y_train = y
    
clf = SVC(C=112, gamma=0.012, kernel='rbf')
clf.fit(X_train, y_train)
print('Final Training Complete')

filename = 'normalizer.pkl'
pickle.dump(normalizer, open(filename,'wb'))
filename = 'svclassifier.pkl'
pickle.dump(clf, open(filename,'wb'))
print('Final models saved')



Final Training Complete
Final models saved
